In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import style
import seaborn as sns
from pylab import rcParams
from tqdm import tnrange, tqdm_notebook, tqdm
import time
plt.style.use('ggplot')
sns.set_style('darkgrid')

import scipy.stats as stats
from scipy.optimize import minimize
import scipy.spatial.distance as dist
import scipy.cluster.hierarchy as sch
from datetime import date
from sklearn.externals import joblib
from sklearn.covariance import shrunk_covariance, ledoit_wolf, OAS, MinCovDet

In [6]:
df = pd.read_csv("returns_data.csv")

In [7]:
df = df.loc[:"December 2017", :]

In [9]:
tickers = ['RPG', 'SVXY', 'TLT', 'GLD', 'UGAZ'] # IV solution

df_px = df[tickers].fillna(method='ffill')

lookback = 21*6
corr_lookback = 21*24
periodicity = 252

leverage = 2
margin_rate = .02768 # <-- quoted IB rate as of December 27, 2017

n_tickers = len(tickers)
N = len(df_px)

df_returns = df_px.pct_change()
df_px.tail()

,RPG,SVXY,TLT,GLD,UGAZ
3027,109.0500,137.43,123.80,125.03,67.9800
3028,109.8200,138.21,124.31,125.44,76.6800
3029,110.6000,138.19,124.52,126.96,82.0400
3030,110.2400,129.74,125.04,127.17,79.6200
3031,111.9277,135.43,125.05,126.96,86.5769


In [10]:
#-------------------------------------
# Weighted arrays
#-------------------------------------
syd_array = np.arange(1, lookback+1)/np.arange(1, lookback+1).sum()
syd_array = syd_array.reshape(-1, 1)
log_array = np.log(np.arange(lookback)+1)/np.log(np.arange(lookback)+1).sum()
log_array = log_array.reshape(-1, 1)
sqrt_array = np.sqrt(np.arange(lookback)+1)/np.sqrt(np.arange(lookback)+1).sum()
sqrt_array = sqrt_array.reshape(-1, 1)

In [20]:
"""
Diversified Risk Parity
Weights are determined so that each eigenvector of the covariance matrix contributes equally to portfolio risk.
"""
def getDiversifiedWeights(w):
    sigma = np.cov(returns.T)
    sigma = shrunk_covariance(sigma, shrinkage=0.05)
    eigvals, eigvecs = np.linalg.eig(sigma)
    eigvals = eigvals.reshape(-1, 1)
    w_tilde = eigvecs.T * w
    v = w_tilde**2 * eigvals
    p = v/v.sum()
    N_Ent = np.exp(-np.sum(p*np.log(p)))
    return -N_Ent

In [21]:
#-------------------------------------
# Ticker selection and lookback input
#-------------------------------------
tickers = ['RPG', 'SVXY', 'TLT', 'GLD', 'UGAZ'] # IV solution

df_px = df[tickers].fillna(method='ffill')

lookback = 21*6
corr_lookback = 21*24
periodicity = 252

leverage = 2
margin_rate = .02768 # <-- quoted IB rate as of December 27, 2017

n_tickers = len(tickers)
N = len(df_px)

df_returns = df_px.pct_change()
df_px.tail()

,RPG,SVXY,TLT,GLD,UGAZ
3027,109.0500,137.43,123.80,125.03,67.9800
3028,109.8200,138.21,124.31,125.44,76.6800
3029,110.6000,138.19,124.52,126.96,82.0400
3030,110.2400,129.74,125.04,127.17,79.6200
3031,111.9277,135.43,125.05,126.96,86.5769


In [33]:
#-------------------------------------
# Weighted arrays
#-------------------------------------
syd_array = np.arange(1, lookback+1)/np.arange(1, lookback+1).sum()
syd_array = syd_array.reshape(-1, 1)
log_array = np.log(np.arange(lookback)+1)/np.log(np.arange(lookback)+1).sum()
log_array = log_array.reshape(-1, 1)
sqrt_array = np.sqrt(np.arange(lookback)+1)/np.sqrt(np.arange(lookback)+1).sum()
sqrt_array = sqrt_array.reshape(-1, 1)

In [35]:
#---------------------------------
# Max diversification inputs and constraints
#---------------------------------
w0 = np.ones([len(tickers), 1])/len(tickers)

cons = ({"type": "eq",
         "fun": lambda x: np.array([1 - np.sum(x)])})

bnds = tuple((0*x-0.0, 0*x+1.0) for x in range(len(tickers)))
#bnds = ((0, 1), (0, 0.1), (0, 1), (0, 1), (0, .1))
# Max diversification
df_returns = df_px.pct_change()

if 'UGAZ' in df_returns.columns:
    df_returns['UGAZ'] *= -1               # Adjust for short sale
    df_returns['UGAZ'] -= .065/periodicity # Adjust for borrow cost

returns_array = np.array(df_returns)

DRP_wts_arr = np.zeros(returns_array.shape) + 1/n_tickers

w0 = np.ones([len(tickers), 1])/len(tickers)

returns = returns_array[1:lookback+1, :]
res = minimize(getDiversifiedWeights, w0, bounds=bnds, constraints=cons, options={"disp": False})
w_temp = res.x.reshape(-1, 1)
DRP_wts_arr[lookback, :] = res.x.reshape(1, -1)

for i in tqdm(range(lookback+1, N)):
    returns = returns_array[i-lookback+1:i+1, :]
    w_temp -= (w_temp - w0)*.20 # adjust beginning weights 20% of the way back to originals
    res = minimize(getDiversifiedWeights, w_temp, bounds=bnds, constraints=cons, options={"disp": False})
    w_temp = res.x.reshape(-1, 1)
    DRP_wts_arr[i] = w_temp.reshape(1, -1)

df_DRP_weights = pd.DataFrame(index=df_returns.index, columns=df_returns.columns, data=DRP_wts_arr)

if 'UGAZ' in df_returns.columns:
    df_returns['UGAZ'] *= -1               # Re-adjust for short sale
    df_DRP_weights['UGAZ'] *= -1
    df_DRP_weights['UGAZ'] = np.minimum(df_DRP_weights['UGAZ'], 0) # UGAZ weight must be negative
    
    # Set weights to max of zero
    df_DRP_weights.loc[:, df_DRP_weights.columns != 'UGAZ'] = np.maximum(
        df_DRP_weights.loc[:, df_DRP_weights.columns != 'UGAZ'], 0)

DRP_returns = (df_DRP_weights.shift(1)*df_returns).sum(axis=1)
df_DRP_weights.tail(1)

/Users/macbook/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log
  del sys.path[0]
/Users/macbook/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in multiply
  del sys.path[0]
100%|██████████| 2905/2905 [01:29<00:00, 32.53it/s]


,RPG,SVXY,TLT,GLD,UGAZ
3031,0.309918,0.052923,0.303486,0.306973,-0.0267
